# 12. Parser de Netlists e Formatos SPICE

Este notebook demonstra como usar o parser de netlists do Pulsim para carregar circuitos de diferentes formatos.

## Tópicos Abordados

1. Formato JSON de netlist
2. Parsing de arquivos e strings
3. Estrutura do netlist
4. Exemplos práticos
5. Integração com simulação

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt
import json
import tempfile
import os

print(f"Pulsim version: {pulsim.__version__}")

## 1. Formato JSON de Netlist

O Pulsim suporta netlists em formato JSON, que é fácil de ler, escrever e manipular programaticamente.

### Estrutura Básica do Netlist JSON

```json
{
    "title": "Nome do Circuito",
    "components": [
        {
            "type": "resistor",
            "name": "R1",
            "nodes": ["n1", "n2"],
            "value": 1000.0
        },
        ...
    ],
    "simulation": {
        "type": "transient",
        "step": 1e-6,
        "stop": 0.01
    }
}
```

## 2. Parsing de Strings JSON

Use `parse_netlist_string()` para criar circuitos a partir de strings JSON.

In [ ]:
# Netlist JSON simples: divisor de tensão
netlist_json = '''
{
    "title": "Divisor de Tensão",
    "components": [
        {
            "type": "voltage_source",
            "name": "V1",
            "nodes": ["vin", "0"],
            "value": 10.0,
            "waveform": "dc"
        },
        {
            "type": "resistor",
            "name": "R1",
            "nodes": ["vin", "vout"],
            "value": 1000.0
        },
        {
            "type": "resistor",
            "name": "R2",
            "nodes": ["vout", "0"],
            "value": 1000.0
        }
    ]
}
'''

# Parse o netlist
circuit = pulsim.parse_netlist_string(netlist_json)
print(f"Circuito carregado: {circuit.node_count()} nós, {circuit.component_count()} componentes")
print(f"Nós: {circuit.node_names()}")

In [ ]:
# Simular o circuito carregado
opts = pulsim.SimulationOptions()
opts.time_step = 1e-6
opts.end_time = 1e-3

result = pulsim.simulate(circuit, opts)

# Verificar tensão no divisor (deve ser 5V)
v_out = result.voltage("vout")
print(f"Tensão de saída (esperado 5V): {v_out[-1]:.2f} V")

## 3. Netlists com Formas de Onda

O formato JSON suporta diferentes tipos de formas de onda para fontes.

In [ ]:
# Netlist com fonte de pulso
pulse_netlist = '''
{
    "title": "Circuito RC com Pulso",
    "components": [
        {
            "type": "voltage_source",
            "name": "Vpulse",
            "nodes": ["vin", "0"],
            "waveform": {
                "type": "pulse",
                "v1": 0.0,
                "v2": 5.0,
                "delay": 0.0,
                "rise": 1e-6,
                "fall": 1e-6,
                "width": 0.5e-3,
                "period": 1e-3
            }
        },
        {
            "type": "resistor",
            "name": "R1",
            "nodes": ["vin", "vout"],
            "value": 1000.0
        },
        {
            "type": "capacitor",
            "name": "C1",
            "nodes": ["vout", "0"],
            "value": 1e-6
        }
    ]
}
'''

circuit_pulse = pulsim.parse_netlist_string(pulse_netlist)

opts = pulsim.SimulationOptions()
opts.time_step = 1e-6
opts.end_time = 5e-3

result = pulsim.simulate(circuit_pulse, opts)

plt.figure(figsize=(10, 4))
plt.plot(result.time * 1000, result.voltage("vin"), label="Entrada (Pulso)")
plt.plot(result.time * 1000, result.voltage("vout"), label="Saída (RC)")
plt.xlabel("Tempo (ms)")
plt.ylabel("Tensão (V)")
plt.title("Circuito RC carregado de Netlist JSON")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Netlist com fonte senoidal
sine_netlist = '''
{
    "title": "Filtro RC Passa-Baixa",
    "components": [
        {
            "type": "voltage_source",
            "name": "Vac",
            "nodes": ["vin", "0"],
            "waveform": {
                "type": "sine",
                "offset": 0.0,
                "amplitude": 1.0,
                "frequency": 1000.0,
                "phase": 0.0
            }
        },
        {
            "type": "resistor",
            "name": "R1",
            "nodes": ["vin", "vout"],
            "value": 1000.0
        },
        {
            "type": "capacitor",
            "name": "C1",
            "nodes": ["vout", "0"],
            "value": 100e-9
        }
    ]
}
'''

circuit_sine = pulsim.parse_netlist_string(sine_netlist)

opts = pulsim.SimulationOptions()
opts.time_step = 1e-6
opts.end_time = 5e-3

result = pulsim.simulate(circuit_sine, opts)

plt.figure(figsize=(10, 4))
plt.plot(result.time * 1000, result.voltage("vin"), label="Entrada")
plt.plot(result.time * 1000, result.voltage("vout"), label="Saída")
plt.xlabel("Tempo (ms)")
plt.ylabel("Tensão (V)")
plt.title("Filtro RC com Fonte Senoidal")
plt.legend()
plt.grid(True)
plt.show()

# Calcular atenuação
fc = 1 / (2 * np.pi * 1000 * 100e-9)  # Frequência de corte
print(f"Frequência de corte: {fc:.1f} Hz")
print(f"Frequência do sinal: 1000 Hz")
att_teorica = 1 / np.sqrt(1 + (1000/fc)**2)
print(f"Atenuação teórica: {att_teorica:.3f}")

## 4. Parsing de Arquivos

Use `parse_netlist_file()` para carregar circuitos de arquivos.

In [ ]:
# Criar um arquivo de netlist temporário
buck_netlist = {
    "title": "Buck Converter Simplificado",
    "components": [
        {
            "type": "voltage_source",
            "name": "Vin",
            "nodes": ["input", "0"],
            "value": 12.0,
            "waveform": "dc"
        },
        {
            "type": "switch",
            "name": "S1",
            "nodes": ["input", "sw", "ctrl"],
            "params": {
                "ron": 0.01,
                "roff": 1e9,
                "vth": 0.5
            }
        },
        {
            "type": "voltage_source",
            "name": "Vctrl",
            "nodes": ["ctrl", "0"],
            "waveform": {
                "type": "pulse",
                "v1": 0.0,
                "v2": 5.0,
                "delay": 0.0,
                "rise": 10e-9,
                "fall": 10e-9,
                "width": 5e-6,
                "period": 10e-6
            }
        },
        {
            "type": "inductor",
            "name": "L1",
            "nodes": ["sw", "output"],
            "value": 100e-6
        },
        {
            "type": "diode",
            "name": "D1",
            "nodes": ["0", "sw"],
            "params": {
                "is": 1e-9,
                "n": 1.5,
                "rs": 0.01
            }
        },
        {
            "type": "capacitor",
            "name": "C1",
            "nodes": ["output", "0"],
            "value": 100e-6
        },
        {
            "type": "resistor",
            "name": "Rload",
            "nodes": ["output", "0"],
            "value": 5.0
        }
    ]
}

# Salvar em arquivo temporário
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump(buck_netlist, f, indent=2)
    temp_path = f.name

print(f"Netlist salvo em: {temp_path}")

# Carregar do arquivo
circuit_file = pulsim.parse_netlist_file(temp_path)
print(f"Circuito carregado: {circuit_file.component_count()} componentes")

# Limpar arquivo temporário
os.unlink(temp_path)

In [ ]:
# Simular o conversor Buck
opts = pulsim.SimulationOptions()
opts.time_step = 50e-9
opts.end_time = 1e-3

result = pulsim.simulate(circuit_file, opts)

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# Tensão de controle
axes[0].plot(result.time * 1e6, result.voltage("ctrl"), 'g-')
axes[0].set_ylabel("V_ctrl (V)")
axes[0].set_title("Conversor Buck - Carregado de Arquivo JSON")
axes[0].grid(True)

# Tensão no switch
axes[1].plot(result.time * 1e6, result.voltage("sw"), 'b-')
axes[1].set_ylabel("V_sw (V)")
axes[1].grid(True)

# Tensão de saída
axes[2].plot(result.time * 1e6, result.voltage("output"), 'r-')
axes[2].set_xlabel("Tempo (us)")
axes[2].set_ylabel("V_out (V)")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Calcular tensão média de saída (após estabilização)
v_out = result.voltage("output")
v_out_avg = np.mean(v_out[-len(v_out)//2:])
duty = 5e-6 / 10e-6  # 50%
v_out_teorico = 12.0 * duty
print(f"\nDuty cycle: {duty*100:.0f}%")
print(f"Tensão de saída média: {v_out_avg:.2f} V")
print(f"Tensão teórica (Vin * D): {v_out_teorico:.2f} V")

## 5. Componentes Suportados no Netlist JSON

### Tipos de Componentes

| Tipo | Descrição | Parâmetros |
|------|-----------|------------|
| `resistor` | Resistor | `value` (Ohms) |
| `capacitor` | Capacitor | `value` (Farads), `v0` (opcional) |
| `inductor` | Indutor | `value` (Henrys), `i0` (opcional) |
| `voltage_source` | Fonte de tensão | `value` ou `waveform` |
| `current_source` | Fonte de corrente | `value` ou `waveform` |
| `diode` | Diodo | `params` (is, n, rs, vt, etc.) |
| `switch` | Chave controlada | `params` (ron, roff, vth) |
| `mosfet` | MOSFET | `params` (vth, kp, type, rds_on, etc.) |
| `transformer` | Transformador | `params` (n, l1, l2, k, etc.) |

In [ ]:
# Exemplo completo com todos os tipos de componentes
complete_netlist = '''
{
    "title": "Circuito Demonstrativo Completo",
    "description": "Exemplo com múltiplos tipos de componentes",
    "components": [
        {
            "type": "voltage_source",
            "name": "Vdc",
            "nodes": ["vcc", "0"],
            "value": 24.0,
            "waveform": "dc"
        },
        {
            "type": "resistor",
            "name": "R1",
            "nodes": ["vcc", "n1"],
            "value": 100.0
        },
        {
            "type": "inductor",
            "name": "L1",
            "nodes": ["n1", "n2"],
            "value": 1e-3,
            "i0": 0.0
        },
        {
            "type": "capacitor",
            "name": "C1",
            "nodes": ["n2", "0"],
            "value": 10e-6,
            "v0": 0.0
        },
        {
            "type": "diode",
            "name": "D1",
            "nodes": ["n2", "out"],
            "params": {
                "is": 1e-9,
                "n": 1.5,
                "rs": 0.01,
                "ideal": false
            }
        },
        {
            "type": "resistor",
            "name": "Rload",
            "nodes": ["out", "0"],
            "value": 50.0
        }
    ]
}
'''

circuit_complete = pulsim.parse_netlist_string(complete_netlist)
print(f"Componentes: {circuit_complete.component_count()}")
print(f"Nós: {circuit_complete.node_names()}")

## 6. Formas de Onda no JSON

### Tipos de Waveform Suportados

| Tipo | Parâmetros |
|------|------------|
| `dc` | `value` |
| `pulse` | `v1`, `v2`, `delay`, `rise`, `fall`, `width`, `period` |
| `sine` | `offset`, `amplitude`, `frequency`, `phase`, `delay` |
| `pwl` | `times`, `values` (arrays) |
| `pwm` | `frequency`, `duty`, `high`, `low`, `deadtime` |

In [ ]:
# Exemplo com fonte PWL (Piecewise Linear)
pwl_netlist = '''
{
    "title": "Circuito com Fonte PWL",
    "components": [
        {
            "type": "voltage_source",
            "name": "Vpwl",
            "nodes": ["in", "0"],
            "waveform": {
                "type": "pwl",
                "times": [0.0, 0.001, 0.002, 0.003, 0.004, 0.005],
                "values": [0.0, 5.0, 5.0, -5.0, -5.0, 0.0]
            }
        },
        {
            "type": "resistor",
            "name": "R1",
            "nodes": ["in", "out"],
            "value": 1000.0
        },
        {
            "type": "capacitor",
            "name": "C1",
            "nodes": ["out", "0"],
            "value": 1e-6
        }
    ]
}
'''

circuit_pwl = pulsim.parse_netlist_string(pwl_netlist)

opts = pulsim.SimulationOptions()
opts.time_step = 1e-6
opts.end_time = 6e-3

result = pulsim.simulate(circuit_pwl, opts)

plt.figure(figsize=(10, 4))
plt.plot(result.time * 1000, result.voltage("in"), label="Entrada (PWL)")
plt.plot(result.time * 1000, result.voltage("out"), label="Saída (Filtrada)")
plt.xlabel("Tempo (ms)")
plt.ylabel("Tensão (V)")
plt.title("Fonte PWL (Piecewise Linear)")
plt.legend()
plt.grid(True)
plt.show()

## 7. Geração Programática de Netlists

Uma vantagem do formato JSON é a facilidade de gerar netlists programaticamente.

In [ ]:
def create_rc_ladder(n_stages, r_value=1000, c_value=1e-6):
    """Cria um netlist de filtro RC ladder com N estágios."""
    components = [
        {
            "type": "voltage_source",
            "name": "Vin",
            "nodes": ["n0", "0"],
            "waveform": {
                "type": "pulse",
                "v1": 0.0,
                "v2": 1.0,
                "delay": 0.0,
                "rise": 1e-9,
                "fall": 1e-9,
                "width": 10e-3,
                "period": 20e-3
            }
        }
    ]
    
    for i in range(n_stages):
        # Resistor
        components.append({
            "type": "resistor",
            "name": f"R{i+1}",
            "nodes": [f"n{i}", f"n{i+1}"],
            "value": r_value
        })
        # Capacitor
        components.append({
            "type": "capacitor",
            "name": f"C{i+1}",
            "nodes": [f"n{i+1}", "0"],
            "value": c_value
        })
    
    return {
        "title": f"RC Ladder Filter ({n_stages} stages)",
        "components": components
    }

# Criar filtros com diferentes números de estágios
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for idx, n_stages in enumerate([1, 2, 3, 4]):
    ax = axes[idx // 2, idx % 2]
    
    # Gerar netlist
    netlist = create_rc_ladder(n_stages, r_value=1000, c_value=100e-9)
    circuit = pulsim.parse_netlist_string(json.dumps(netlist))
    
    # Simular
    opts = pulsim.SimulationOptions()
    opts.time_step = 1e-6
    opts.end_time = 2e-3
    
    result = pulsim.simulate(circuit, opts)
    
    # Plotar
    ax.plot(result.time * 1000, result.voltage("n0"), 'b--', alpha=0.5, label="Entrada")
    ax.plot(result.time * 1000, result.voltage(f"n{n_stages}"), 'r-', label="Saída")
    ax.set_xlabel("Tempo (ms)")
    ax.set_ylabel("Tensão (V)")
    ax.set_title(f"RC Ladder - {n_stages} estágio(s)")
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()

## 8. Validação e Tratamento de Erros

O parser realiza validações e retorna erros descritivos.

In [ ]:
# Exemplo de netlist com erro
invalid_netlists = [
    # JSON inválido
    '{"title": "Teste", "components": [}',
    
    # Tipo de componente desconhecido
    '{"components": [{"type": "unknown_type", "name": "X1", "nodes": ["a", "b"]}]}',
    
    # Faltando campo obrigatório
    '{"components": [{"type": "resistor", "nodes": ["a", "b"]}]}',  # sem "name"
]

for i, netlist in enumerate(invalid_netlists):
    print(f"\nTestando netlist inválido {i+1}:")
    try:
        circuit = pulsim.parse_netlist_string(netlist)
        print("  Sucesso (inesperado)")
    except Exception as e:
        print(f"  Erro capturado: {type(e).__name__}")
        print(f"  Mensagem: {str(e)[:80]}..." if len(str(e)) > 80 else f"  Mensagem: {e}")

## 9. Exemplo Prático: Retificador de Meia Onda

Vamos criar um circuito mais complexo usando netlist JSON.

In [ ]:
rectifier_netlist = {
    "title": "Retificador de Meia Onda com Filtro",
    "components": [
        {
            "type": "voltage_source",
            "name": "Vac",
            "nodes": ["ac_in", "0"],
            "waveform": {
                "type": "sine",
                "offset": 0.0,
                "amplitude": 170.0,  # 120Vrms * sqrt(2)
                "frequency": 60.0,
                "phase": 0.0
            }
        },
        {
            "type": "resistor",
            "name": "Rs",  # Resistência da fonte
            "nodes": ["ac_in", "ac_out"],
            "value": 1.0
        },
        {
            "type": "diode",
            "name": "D1",
            "nodes": ["ac_out", "dc_out"],
            "params": {
                "is": 2.55e-9,  # 1N4007
                "n": 1.75,
                "rs": 0.042,
                "ideal": False
            }
        },
        {
            "type": "capacitor",
            "name": "Cfilt",
            "nodes": ["dc_out", "0"],
            "value": 470e-6  # 470uF
        },
        {
            "type": "resistor",
            "name": "Rload",
            "nodes": ["dc_out", "0"],
            "value": 1000.0  # 1kOhm
        }
    ]
}

# Parse e simular
circuit_rect = pulsim.parse_netlist_string(json.dumps(rectifier_netlist))

opts = pulsim.SimulationOptions()
opts.time_step = 10e-6
opts.end_time = 100e-3  # 6 ciclos

result = pulsim.simulate(circuit_rect, opts)

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Tensão AC e DC
axes[0].plot(result.time * 1000, result.voltage("ac_in"), 'b-', alpha=0.7, label="Entrada AC")
axes[0].plot(result.time * 1000, result.voltage("dc_out"), 'r-', label="Saída DC")
axes[0].set_ylabel("Tensão (V)")
axes[0].set_title("Retificador de Meia Onda - Carregado de Netlist JSON")
axes[0].legend()
axes[0].grid(True)

# Zoom na saída para ver ripple
# Mostrar últimos 2 ciclos
mask = result.time > 0.066
axes[1].plot(result.time[mask] * 1000, result.voltage("dc_out")[mask], 'r-')
axes[1].set_xlabel("Tempo (ms)")
axes[1].set_ylabel("Tensão DC (V)")
axes[1].set_title("Detalhe do Ripple")
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Calcular ripple
v_dc = result.voltage("dc_out")[mask]
v_max = np.max(v_dc)
v_min = np.min(v_dc)
v_avg = np.mean(v_dc)
ripple_pp = v_max - v_min
ripple_percent = (ripple_pp / v_avg) * 100

print(f"\nAnálise do Retificador:")
print(f"  Tensão máxima: {v_max:.1f} V")
print(f"  Tensão mínima: {v_min:.1f} V")
print(f"  Tensão média DC: {v_avg:.1f} V")
print(f"  Ripple pico-a-pico: {ripple_pp:.1f} V")
print(f"  Ripple: {ripple_percent:.1f}%")

## 10. Resumo

### Funções de Parsing

| Função | Descrição |
|--------|----------|
| `parse_netlist_file(path)` | Carrega circuito de arquivo JSON |
| `parse_netlist_string(json_str)` | Carrega circuito de string JSON |

### Vantagens do Formato JSON

1. **Legibilidade**: Fácil de ler e editar manualmente
2. **Programático**: Fácil de gerar e manipular com código
3. **Flexível**: Suporta todos os componentes e waveforms do Pulsim
4. **Validação**: Parser retorna erros descritivos
5. **Portabilidade**: Formato padrão suportado por todas as linguagens

### Próximos Passos

- Explorar a biblioteca de dispositivos pré-definidos
- Usar o parser SPICE para compatibilidade com outras ferramentas
- Integrar com ferramentas de automação e otimização